![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidget_2 Humidity and Temperature

This short notebook builds on how we access a Phidgets device from a Jupyter notebook.

We will be modifying code from Phidgets_1 in order to get both temperature and humidity to display on the screen.

It is important to run this notebook in the Google Chrome web browser, on a desktop or laptop computer. Unfortunately, other browsers (FireFox, Safari, Edge) will not work as they cannot access the USB port that connects to the Phidget device. That may change in the future, but at the time of writing, this is currently only possible in Chrome.

<h2 style="color: red">IMPORTANT</h2>
You MUST click the <span style="color:red">BIG, RED DISCONNECT BUTTON</span> at the end of this notebook when you are done. This tells the software to release the Phidget device, so it can be used by other notebooks you might try later.

### Setting up the Phidget device

We will be using the Phidget Starter Kit, as shown in this figure:
<img src="images/KIT4003.jpg" alt="Image of the starter kit"  width = 500 />

Verify these points:
- Get a hold of a Phidgets Starter Kit. It should already be assembled. If not, assemble it now.
    - You will have received instructions with the kit on how to assemble it.
- The Starter Kit has a humidty sensor attached. If it does not, find the sensor and attach it.
    - The humidty senor is a small black box attached with a thin cable. It is marked with the initials "HUM."
- Connect the Starter Kit to your computer with a USB cable. 
    - The Kit come with a USB cable, it is attached to the Hub device inside the Starter Kit.
- If you have an older version of the Phidgets, you may need to update its firmware. See the following for details:
    - https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/
- As mentioned above, make sure you are running this notebook in the Google Chrome browser. 


## Overview of the code.

It's a bit tricky to use the Phidget device, as we have to "tell" the computer explicity that we want to use the Phidget. This means writing code that tell the computer that, first, we want to use the USB port and cable to connect to the device, and then we want to use a specific device in the starter kit, and finally we want to read data from that deive. 

We will do this in a step-by-step manner, so we can understand how the connection process works. At the end, it is always good practice to clean things up by closing all the connections. 

The main steps are thus:

0. Import some Python libraries
1. Import the Phidget library, in Javascript
2. Define and open the USB connection
3. Open the Humidity and Temperature devices
4. Read some values from the devices
5. When we are done, close the devices and USB connection

Now let's get into the details

## Step 0.
Let's import a few Python libraries that we need. 

In [ ]:
import ipywidgets as widgets
from IPython.display import Javascript, display
from time import sleep

## Step 1.

We load in the Phidgets22 library of code in JavaScript.

In [ ]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

In [ ]:
## We pause for a second here, to allow the library to load
sleep(1)

## Step 2.
We now open a USB connection.  This Javascript code requests the USB port to open and access the Phidget Hub. 

In [ ]:
%%js

if (window.usbconn === undefined) {
    
    element.text("Creating a new USB Connection.");
    
    window.usbconn = new phidget22.USBConnection();
    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

In [ ]:
## We rest for a bit while the USB port opens
sleep(1)

## Step 2 - Confirm.

Let's check that the connection is open. If it is not, then try running the cell above again. 

In [ ]:
%%js
element.text(usbconn.connected);

## Step 3.

Let's open both the humidity and temperature sensors. Notice how the open codes are very similar for the two sensors. 

In [ ]:
%%js
window.humSensor = new phidget22.HumiditySensor();
window.tempSensor = new phidget22.TemperatureSensor();
humSensor.open();
tempSensor.open();

In [ ]:
## We rest for a bit while the devices connect.
sleep(1)

### Check

Let's check that the sensors were opened, by running the following code. 

If you get a "false," try running the cell above again. 

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + humSensor.isOpen);
element.append("<p> Is the temperature sensor attached? " + tempSensor.isOpen);

## Step 4.  

Reading the value for the device

We saw in an earlier notebook that we can read the humidity value with the following.

In [ ]:
%%js
element.text(humSensor.humidity);

## Try it yourself - temperature
How would you modify the cell above to read the temperature value?

Hint: you need to change both the name of sensor (humSensor) and the output variable (humidity).
    
Try out your own code in the cell below. 

In [ ]:
%%js
element.text("Put your code in this cell.")


## Step 4+. Displaying several values

Let's write a loop in Python to display several readings in sequence. We will import some helper function for the IPython module.

Also, we can make the display more informative by adding some words to the numerical reading. 


In [ ]:
from IPython.display import Javascript, display
from time import sleep

for i in range(10):
    display(Javascript("""
    element.text("Humdity is " + humSensor.humidity + "%. ");
    element.append("Temperature is " + tempSensor.temperature + " degrees.");
    """))
    sleep(.25)

## Step 5. Closing down the sensors. 

Once we are all done, we should close the connection to the phidget. This will leave it free for the next person to use in their experiments. 

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook. 

In [ ]:
def doDisconnect(b):
    display(Javascript("""
        (async () => {
            await humSensor.close();
            await tempSensor.close();
            usbconn.close();
            usbconn.delete();
            delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
        })();
    """))

run_button = widgets.Button(
    description = 'IMPORTANT: Click to disconnect', 
        button_style='danger',layout=widgets.Layout(width='50%', height='80px')
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)

### Confirm

You can confirm the phidget is open or closed by running the following cell. 

If it is open (true), try clicking the button above, again. 

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + humSensor.attached);
element.append("<p> Is the temperature sensor attached? " + tempSensor.attached);


## In case of errors

What kind of errors can happen? What should they do about them?

Symptom 1. At Step 2, you keep getting "Connection Errors."
- Problem is likely that you are not using the Chrome browser.
    - Solution is to switch to Chrome.
- You might have an older version of a Phidget that needs a firmware update.
    - See here for the solution: https://www.phidgets.com/education/educators/advanced-troubleshooting/firmware-upgrade/
    
Symptom 2. At Step 2, the requestWebUSBDeviceAccess() command fails.
- Check that you see an image like this: <img src="images/Pconnect.png" alt="Hey"  width = 300 />
- If you don't see it, this probably means your Phidget Starter Kit is not connected.
- Solution is to plug in the Phidget to your computer's USB port, then try again.
    
    
Symptom 3. At Step 2, the image above appears, but you do not see the VINT device displayed.
- Problem is likely that the Phidget device is not attached to your computer.
- Solution is to plug in the Phidget to your computer's USB port, then try again.



Symptom 4. At Step 2 or 3, the image above appears and you select the VINT device, but nothing else happens.
- Problem is likely that another piece of software is already connected to the device. For instance, if you have another Jupyter notebook using the device, or are using Phidget's MakeCode software.
- Solution is to stop the other software from running.
- In some cases, you might need to shut down all the notebooks and log out of the Callysto Hub. Then log back in and start anew. 
- don't forget to click the <span style="color:red">BIG, RED DISCONNECT BUTTON</span> above when you are done. 

Symptom 5. JavaScript errors occur, like **TypeError: Cannot read properties of undefined (reading 'attached')**
- check the spelling of the variables and commands

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)